In [1]:
# Library for opening url and creating requests
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd

In [2]:
# Opens a website and read its binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its binary contents (HTTP Response Body)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    #making request to the website
    req = urllib.request.Request(url=url,headers=headers)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [3]:
# Converting Top 2 rows as header
def header_convert(df):
    # Combine the top 2 rows into a single header row
    new_header = df.iloc[0].fillna('')  
    df = df[1:]  # Exclude the first two rows

    # Assign the combined row as the new header
    df.columns = new_header

    # Reset the DataFrame index if needed
    df.reset_index(drop=True, inplace=True)
    
    return df

In [4]:
def generate_url(fund_name, start_date, end_date):
    # Replace spaces with %20 for URL encoding
    fund_name_encoded = fund_name.replace(" ", "%20")
    
    # Construct the URL
    base_url = "https://www.advisorkhoj.com/mutual-funds-research/historical-NAV/"
    url = f"{base_url}{fund_name_encoded}?start_date={start_date}&end_date={end_date}"
    
    return url

# Example usage
fund_name = "Axis BlueChip Dir Gr"
fund_name_1 = "Parag Parikh Arbitrage Fund Dir Gr"
start_date = "16-08-2023"
end_date = "16-09-2024"

Link = generate_url(fund_name, start_date, end_date)

In [5]:
# defining the html contents of a URL.
xhtml = url_get_contents(Link).decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the HTMLTableParser object
p.feed(xhtml)

In [6]:
# Shots Summary
df = pd.DataFrame(p.tables[0])
df = header_convert(df)

In [7]:
# Convert 'NAV Date' column to datetime
df['NAV Date'] = pd.to_datetime(df['NAV Date'], format='%d-%m-%Y')

# Ensure 'NAV (Rs)' is numeric
df['NAV (Rs)'] = pd.to_numeric(df['NAV (Rs)'], errors='coerce')

# Extract month-year and day
df['month_year'] = df['NAV Date'].dt.strftime('%b-%y')  # Month-Year format
df['day'] = df['NAV Date'].dt.day

# Pivot table to get month_year as columns and days as rows
pivot_table = df.pivot(index='day', columns='month_year', values='NAV (Rs)')

# Create a sorted list of month-year columns
date_range = pd.date_range(start=df['NAV Date'].min(), end=df['NAV Date'].max(), freq='MS')
sorted_columns = [d.strftime('%b-%y') for d in date_range]

# Reindex columns to ensure correct order
pivot_table = pivot_table.reindex(columns=sorted_columns)
pivot_table = pivot_table.reset_index()
original_table = pivot_table.copy()

In [8]:
# Scenario 1
pivot_table['Average'] = pivot_table.drop(columns='day').mean(axis=1)
pivot_table.columns.name = None

max_average = pivot_table['Average'].max()
pivot_table['Average_pct_diff'] = (max_average - pivot_table['Average']) / max_average * 100

In [9]:
# Scenario 2
pivot_table2 = original_table.bfill()
pivot_table2['Average'] = pivot_table2.drop(columns='day').mean(axis=1)
pivot_table2.columns.name = None

max_average2 = pivot_table2['Average'].mean()
pivot_table2['Average_pct_diff'] = (max_average2 - pivot_table2['Average']) / max_average2 * 100

In [10]:
# Function to create color shading for positive values
def color_positive(val, min_val, max_val):
    if pd.isna(val):
        return ''
    
    # Normalize the value
    normalized = (val - min_val) / (max_val - min_val) if max_val != min_val else 0
    
    # Color shading for positive values
    red = 255 - int(normalized * 80)  # Light green: Red component decreases
    green = 255
    blue = 255 - int(normalized * 80)  # Light green: Blue component decreases
    return f'background-color: rgb({red}, {green}, {blue})'

# Function to create color shading for negative values
def color_negative(val, min_val, max_val):
    if pd.isna(val):
        return ''
    
    # Normalize the value
    normalized = (val - min_val) / (max_val - min_val) if max_val != min_val else 0
    
    # Color shading for negative values
    red = 255
    green = 255 - int(normalized * 80)  # Light red: Green component decreases
    blue = 255 - int(normalized * 80)  # Light red: Blue component decreases
    return f'background-color: rgb({red}, {green}, {blue})'

# Apply the style to the 'Average_pct_diff' column
def apply_color_map(df):
    min_val = df['Average_pct_diff'].min()
    max_val = df['Average_pct_diff'].max()
    
    # Apply color shading based on whether the value is positive or negative
    return df.style.applymap(lambda val: color_positive(val, min_val, max_val) if val >= 0 else color_negative(val, min_val, max_val), subset=['Average_pct_diff'])

# Get the styled DataFrame
styled_df = apply_color_map(pivot_table2)

# Display styled DataFrame (useful in Jupyter Notebook)
styled_df


C:\Users\Pranav Jaju\AppData\Local\Temp\ipykernel_27880\3378717283.py:35: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return df.style.applymap(lambda val: color_positive(val, min_val, max_val) if val >= 0 else color_negative(val, min_val, max_val), subset=['Average_pct_diff'])


,day,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24,Feb-24,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Average,Average_pct_diff
0,1,52.130000,52.660000,51.410000,54.590000,58.120000,58.020000,60.580000,62.500000,62.880000,64.050000,66.970000,68.470000,70.070000,60.188462,0.227661
1,2,52.210000,52.660000,51.770000,55.540000,57.970000,58.440000,60.620000,62.440000,62.880000,64.050000,66.800000,67.980000,70.070000,60.263846,0.102699
2,3,52.210000,52.660000,52.040000,55.540000,57.620000,58.230000,60.620000,62.340000,62.480000,64.050000,67.190000,66.310000,70.070000,60.104615,0.366650
3,4,52.210000,52.460000,52.520000,55.540000,58.160000,58.230000,60.620000,62.710000,62.400000,60.980000,67.240000,66.310000,69.980000,59.950769,0.621676
4,5,52.380000,52.830000,52.520000,55.870000,58.200000,58.230000,60.390000,62.670000,62.400000,63.100000,67.350000,66.310000,70.070000,60.178462,0.244238
5,6,52.560000,53.270000,52.520000,56.220000,57.760000,58.720000,60.700000,63.130000,62.400000,63.510000,67.180000,66.060000,69.450000,60.267692,0.096323
6,7,52.730000,52.870000,52.470000,56.130000,57.760000,58.820000,60.950000,63.130000,61.960000,64.760000,67.180000,67.130000,69.770000,60.435385,-0.181655
7,8,53.010000,52.870000,52.600000,56.210000,57.760000,58.370000,60.840000,63.130000,62.130000,64.820000,67.180000,66.720000,69.770000,60.416154,-0.149776
8,9,53.350000,52.870000,52.590000,56.390000,57.970000,58.570000,60.840000,63.130000,61.400000,64.820000,67.450000,67.340000,69.770000,60.499231,-0.287490
9,10,53.350000,53.290000,52.630000,56.390000,58.100000,58.180000,60.840000,63.390000,61.780000,64.820000,67.140000,67.200000,70.000000,60.546923,-0.366548
